# Multi Agent - Agentic AI

### import required libraries

In [ ]:
import os
import openai
from agents import Agent, Runner, function_tool
import asyncio

In [ ]:
os.environ["OPENAI_BASE_URL"] = os.getenv("LMSTUDIO_API_BASE_URL", "http://localhost:1234/v1")
os.environ["OPENAI_API_KEY"] = os.getenv("LMSTUDIO_API_KEY", "lm-studio")

openai_model = "openai/gpt-oss-20b"

### Agents

#### Agent: Use case Extractor based on the scenario

In [ ]:
agent_usecase_extractor_instruction = """
You are an AI Use Case Extractor specializing in Life & Annuity insurance.

GOAL
From the given business scenario, identify and return the SINGLE most valuable AI use case.
“Best” means the highest combined impact across efficiency, risk reduction, customer experience,
and feasibility in a regulated Life & Annuity environment.

WHAT TO DO
1) Briefly understand the scenario and underlying business pain points.
2) Identify all possible AI opportunities mentally, but SELECT ONLY ONE.
3) Choose the use case that:
   - Solves the most critical bottleneck or risk
   - Delivers measurable business value
   - Is realistic to implement (not speculative)
4) Prefer use cases that improve compliance, reduce manual effort, and enhance accuracy.

OUTPUT FORMAT (STRICT – ONE USE CASE ONLY)

- Use case name:
- Category: (Customer | Agent Assist | Ops Automation | Risk/Compliance)
- One-liner: What the use case does and why it matters
- Trigger / Input:
- Data required:
- AI approach:
- Business impact (measurable KPIs):
- Feasibility: (High / Medium / Low) with reason
- Key risks & controls:
- MVP scope (2–4 weeks):

CONSTRAINTS
- Output ONLY ONE use case.
- Do NOT list alternatives or comparisons.
- Do NOT add preamble, explanation, or conclusion.
- Assume PII-sensitive, regulated insurance operations.
- Use plain business language; avoid buzzwords.

QUALITY BAR
This output should be clear enough to:
- Put directly into a business case
- Share with compliance and architecture teams
- Use as a Phase-1 AI pilot candidate
"""

agent_usecase_extractor = Agent(
    name="AI Use Case Extractor",
    instructions=agent_usecase_extractor_instruction,
    model=openai_model,
)

In [ ]:
agent_usecase_extractor_input_text = """
The Life Insurance call center handles customer service calls.
Agents validate customer identity, update bank account details,
check policy payment status, and updates the ACH payments.
The process is manual, time-consuming, and prone to errors,
especially during bank account change requests.
"""

In [ ]:
result_agent_usecase_extractor = Runner.run_streamed(agent_usecase_extractor, input=agent_usecase_extractor_input_text)
async for event in result_agent_usecase_extractor.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
usecases = result_agent_usecase_extractor.final_output

#### Agent: Email created in HTML format

In [ ]:
agent_email_creator_html_format_instruction = """
You are an AI Email Creation Agent specializing in professional HTML emails.

GOAL
Create a complete, ready-to-send email in clean, well-structured HTML.

RESPONSIBILITIES
- Select a clear and appropriate email subject line
- Write a concise, professional email body in valid HTML
- Determine the appropriate sender ("From") and recipient ("To") email addresses
  based on the provided context
- Maintain a professional business tone unless specified otherwise

OUTPUT REQUIREMENTS
- Output MUST be valid HTML for the email body
- Subject line must be provided separately (plain text)
- Do NOT include explanations, comments, or markdown
- Do NOT include CSS files or scripts (inline HTML only)
- Ensure readability across common email clients (Gmail, Outlook)

ASSUMPTIONS & SAFETY
- Assume business communication standards
- Avoid sensitive data leakage

OUTPUT FORMAT (STRICT)
Subject:
HTML Body:
"""

In [ ]:
agent_email_creator_html_formator = Agent(name="Email Creator in HTML format", 
                                          instructions=agent_email_creator_html_format_instruction,
                                          model=openai_model
)

In [ ]:
email_contacts = {
    "to:": "blitznihar@gmail.com", 
    "from": "transformationleader@gmail.com"
    }

result_agent_email_creator_html_formator = Runner.run_streamed(agent_email_creator_html_formator, input=usecases)

async for event in result_agent_email_creator_html_formator.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
            
usecase_email = result_agent_email_creator_html_formator.final_output

### Tools

#### Send Email Tools

In [ ]:
@function_tool
def send_email(body: str, emailto: str, emailfrom: str, subject: str):
    host = os.getenv("debugmailhost")
    port = int(os.getenv("debugmailport"))
    login = os.getenv("debugmaillogin")
    password = os.getenv("debugmailpassword")
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    import smtplib 
    msg = MIMEMultipart()
    msg['From'] = emailfrom
    msg['To'] = emailto
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'html'))
    server = smtplib.SMTP(host, port)
    server.starttls()
    server.login(login, password)
    text = msg.as_string()
    server.sendmail(msg['From'], msg['To'], text)
    server.quit()
    return {"status": "success"}

#### Use Case Creator tool

In [ ]:
agent_usecase_extractor_tool = agent_usecase_extractor.as_tool(tool_name="Use Case Extractor based on the scenarios", tool_description="Extracts the most valuable AI use case from a given business scenario in Life & Annuity insurance.")
agent_email_creator_html_formator_tool = agent_email_creator_html_formator.as_tool(tool_name="Email Creator in HTML format", tool_description="Creates a professional HTML email based on the provided use case analysis.")  

In [ ]:
FinalAgentInstruction = "you analyse the use case based on scenario and create the final email body with the complete analysis report and send email to the provided email contact"
FinalAgent = Agent(name="AI Use case analysis and email body creator and formatter and send the email using send_email tool",
               instructions=FinalAgentInstruction,
               model=openai_model,
               tools=[agent_usecase_extractor_tool, agent_email_creator_html_formator_tool, send_email])
scenario = agent_usecase_extractor_input_text
finalAgentInput = f"Create the final email body with the complete analysis report including identified use cases based on the scenario: {scenario} and send email to email contact: {email_contacts}"
final_result = Runner.run_streamed(FinalAgent, 
                                input=finalAgentInput)

async for event in final_result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)
            
# print("Final Result:", final_result.final_output)

In [ ]:
FinalAgent = Agent(
    name="AI Use case analysis and email body creator and sender",
    instructions=FinalAgentInstruction,
    model=openai_model,
    tools=[agent_usecase_extractor_tool, agent_email_creator_html_formator_tool, send_email],
)

# 5) Build clean input
scenario = agent_usecase_extractor_input_text

email_contacts = {
    "to": "blitznihar@gmail.com",
    "from": "transformationleader@gmail.com"
}

finalAgentInput = f"""
Scenario:
{scenario}

Send email with:
From: {email_contacts["from"]}
To: {email_contacts["to"]}
""".strip()


# 6) Stream correctly (IMPORTANT: await)
final_result = Runner.run_streamed(FinalAgent, input=finalAgentInput)

async for event in final_result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
        print(event.data.delta, end="", flush=True)

In [ ]:
FinalAgent

### Agent Handover

#### Handover: Email Service

In [ ]:
handoff = [send_email]
FinalAgentInstruction = "you analyse the use case based on scenario and create the final email body with the complete analysis report and send email to the provided email contact"
FinalAgent = Agent(name="AI Use case analysis and email body creator and formatter and send the email using send_email tool",
               instructions=FinalAgentInstruction,
               model=openai_model,
               tools=[agent_usecase_extractor_tool, agent_email_creator_html_formator_tool],
               handoffs=handoff,
               handoff_description="send the email using send_email tool"
               )
email_contacts = {
    "to": "blitznihar@gmail.com",
    "from": "transformationleader@gmail.com"
}
finalAgentInput = f"""scenario:
{scenario}
email_contacts:
{email_contacts}
"""

            
FinalAgent

In [ ]:
final_result = Runner.run_streamed(FinalAgent, input=finalAgentInput)
async for event in final_result.stream_events():
    if event.type == "raw_response_event" and event.data.type == "response.output_text.delta":
            print(event.data.delta, end="", flush=True)